<a href="https://colab.research.google.com/github/laNaranjaja/GenAI/blob/main/My_Screening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install -q streamlit
! pip install Openai
! pip install langchain
! pip install replicate
! pip install pypdf
! npm install localtunnel

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.4/196.4 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 32.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.1/226.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.4 MB/s eta 0:00:00
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.5.0:
      Successfully uninstalled typing_extensions-4.5.0
ERROR: pip's dependency resolver does not currently take into account all the pac

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
%%writefile app.py
import streamlit as st
import os
from langchain.chat_models import ChatOpenAI
from langchain.llms import Replicate
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models import ChatVertexAI
from pypdf import PdfReader
import json
import pandas as pd

def getpdftext(archivo):
  pdfreader = PdfReader(archivo)
  text = ''
  for page in pdfreader.pages:
    text += page.extract_text()
  return text

def llm_extract(texto):
  prompt='''
  You are a useful HR assistant that looks certain
   information and returns a given format. For the provided data, look for the following fields and
   return them if available. Always return data in JSON format only. Do not reply with any other format nor add any extra details or text to the output
   DO NOT REPLY WITH BACKTICKS NOR INCLUDE THE WORD 'json' AS PART OF YOUR RESPONSE
   EXPECTED RESPONSE:
   {
    "Candidate Name":<The candidates name. "None" if unavailable or not provided>,
    "Candidate Phone":<The candidate's phone number. "None" if unavailable or not provided>,
    "Candidate Email":<The candidate's email number. "None" if unavailable or not provided>,
    "Languages":<A list of the languages the candidate can speak>,
    "Major Studies":<Candidate's major studies. "None" if unavailable or not provided>,
    "Master Studies":<Candidate's master's studies. "None" if unavailable or not provided>,
    "PhD Studies":<Candidate's PhD studies. "None" if unavailable or not provided>,
    "AWS Experience":<"True" if the candidate has any experience or certification related to AWS. "False" if unavailable or not provided. Only reply with either True or False, but nothing else>,
    "GCP Experience":<"True" if the candidate has any experience or certification related to Google Cloud. "False" if unavailable or not provided. Only reply with either True or False, but nothing else>,
    "Cisco Experience":<"True" if the candidate has any experience or certification related to Cisco. "False" if unavailable or not provided. Only reply with either True or False, but nothing else>,
    "Cybersecurity Experience":<"True" if the candidate has any experience or certification related to Cybersecurity. "False" if unavailable or not provided. Only reply with either True or False, but nothing else>,
    "Candidate Summary":<Parahraph summarizing the infor that corresponds to the candidate>
   }
   BELOW IS AN EXAMPLE OF A GOOD OUTPUT:
   {
    "Candidate Name":"Hugo Martinez,
    "Candidate Phone":"+52 55 77889966",
    "Candidate Email":"hugo@gmail.com",
    "Languages":"English, Spanish, German"
    "Major Studies:"Chemical Engineering",
    "Master Studies":"Biomedics",
    "PhD Studies":"Biostatistics",
    "AWS Experience":"False",
    "GCP Experience":"True",
    "Cisco Experience":"False",
    "Cybersecurity Experience":"False",
    "Candidate Summary":"Hugo Martinez is an accomplished candidate with a diverse academic background and expertise in the field of science and engineering. He holds a Bachelor's degree in Chemical Engineering, showcasing a strong foundation in the fundamentals of the discipline. Building upon this, Hugo pursued his Master's studies in Biomedics, demonstrating a keen interest in the intersection of engineering and medical sciences. Furthermore, he has expanded his academic horizons with a PhD in Biostatistics, emphasizing a solid understanding of statistical methods in the realm of life sciences. Notably, Hugo brings a technological edge to his skill set with experience in Google Cloud Platform (GCP). Although he doesn't possess AWS, Cisco, or Cybersecurity experience according to the provided data, his existing proficiency in GCP suggests a readiness to adapt and leverage cutting-edge technologies. With a well-rounded educational background and a strategic focus on technology, Hugo Martinez stands out as a promising candidate with the potential to contribute meaningfully to a variety of professional roles"
   }

  '''
  query=texto

  result = chat_gcp(
          [
              SystemMessage(content=prompt),
              HumanMessage(content=query)
          ])
  return json.loads(result.content)


st.write("CV Data Extractor Using PaLM 📶")
archivos_pdf = st.file_uploader('Sube aqui los CVs en formato PDF', type=['pdf'], accept_multiple_files=True)

#Modelo PALM2 text-bison
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/gdrive/MyDrive/GenAIEne2024/unified-surface-410604-c94c40d752ce.json'
chat_gcp = ChatVertexAI(max_output_tokens=2048, temperature=0)

datos = []

if archivos_pdf:
  with st.spinner('Generando respuesta con PALM 2...'):
    for archivo in archivos_pdf:
      try:
        dato = llm_extract(getpdftext(archivo))
        datos.append(dato)
      except Exception as e:
        st.write(f"Error: {e}")
    df = pd.DataFrame(datos)
    st.dataframe(df)
    st.balloons()

Overwriting app.py


In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

Password/Enpoint IP for localtunnel is: 34.73.83.211


In [ ]:
!streamlit run app.py &>/content/logs.txt &
!npx localtunnel --port 8501

npx: installed 22 in 3.341s
your url is: https://fruity-monkeys-brush.loca.lt
^C
